In [1]:
import numpy as np
from pprint import pprint

In [2]:
def fixed_params(principle_amount, interest_rate, tenure):
    loan_variables = []
    emi = []
    no_of_loans = len(principle_amount)
    for j in range(no_of_loans):
        interest_rate_per_month = interest_rate[j] /(12 *100)
        emi.append((principle_amount[j] * interest_rate_per_month * (1 + interest_rate_per_month)**tenure[j]) / ((1 + interest_rate_per_month)**tenure[j] - 1))
        OB = []  # Principal outstanding
        P = []  # Monthly principal
        I = []  # monthly interest
        CB = [] # Closing balance of the month
        for i in range(tenure[j]):
            if i == 0:
                OB.append(principle_amount[j])
            else:
                OB.append(CB[i-1])
            I.append(OB[i] * interest_rate_per_month)
            P.append(emi[j] - I[i])
            CB.append(OB[i] - P[i])
        loan_variables.append({'OB': OB, 'P': P, 'I': I, 'CB': CB})
    return emi, loan_variables, no_of_loans

In [3]:
def monthly_data(loan_variables, no_of_loans, months_paid, tenure, total_saved_yet, future_months_saving_plan, fixed_monthly_saving):
    monthly_bal_list, monthly_interest, savings_list, closed_loans, results = [], [], [], [], []
    index = [0 ] *no_of_loans
    stop = 0
    max_months_paid = max(months_paid)
    total_no_of_payment_months = max_months_paid + max([tenure[i] - months_paid[i] for i in range(no_of_loans)])
    for i in range(total_no_of_payment_months+1):
        month_bal = [0] * no_of_loans
        interest_list = [0] * no_of_loans
        for j in range(no_of_loans):
            if i > max_months_paid -months_paid[j] -1 and i < max_months_paid -months_paid[j] +tenure[j]:
                month_bal[j] = (loan_variables[j]['OB'][index[j]])
                interest_list[j] = (loan_variables[j]['I'][index[j]])
                index[j] += 1
        monthly_interest.append(interest_list)
        monthly_bal_list.append(month_bal)
        if i == 0:
            saving = total_saved_yet
        else:
            saving = savings_list[i-1]
        if i > max_months_paid:
            if i- max_months_paid in future_months_saving_plan:
                saving += future_months_saving_plan[i - max_months_paid]
                saving += fixed_monthly_saving
            else:
                saving += fixed_monthly_saving
        savings_list.append(saving)
    return monthly_bal_list, monthly_interest, savings_list, total_no_of_payment_months, max_months_paid

In [4]:
def my_arg(aa, min_or_max, c):
    a = aa
    if min_or_max == 'min':
        for x in c:
            a[x] = 100000000000000
        return np.argmin(a)
    else:
        for x in c:
            a[x] = -10000000000000
        return np.argmax(a)

In [15]:
def analysis(max_months_paid, monthly_bal_list, monthly_interest, savings_list, total_no_of_payment_months, no_of_loans, months_paid, criterion, foreclosure_charge_percent, principle_amount):
    closed_loans, results = [], []
    for i in range(len(monthly_bal_list)):
        for l in range(no_of_loans):
            if monthly_bal_list[i][l] <1 and monthly_bal_list[i-1][l] >1 and l not in closed_loans:
                results.append({'comments': 'auto closure done in this month', '#loan': l, 'cumulative saving': savings_list[i], 'payment amount': 0, 'closed loan of amount': principle_amount[l], 'after #months': i - max_months_paid})
                closed_loans.append(l)
                
        study_loans, marked_loans = [], []
        for l in (set(range(no_of_loans)) - set(closed_loans)):
            settle_amount = monthly_bal_list[i][l] * (1 + (foreclosure_charge_percent[l] / 100))
            if savings_list[i] > settle_amount:
                study_loans.append(l)
                
        if len(study_loans) > 0:
            study_loan = my_arg(monthly_interest[i], 'max', set(range(no_of_loans)) - set(study_loans))
#             print(study_loans)
            study_loans.remove(study_loan)
            marked_loans.append(study_loan)
            results.append({'comments': 'preclosure', '#loan': study_loan, 'cumulative saving': savings_list[i], 'payment amount': monthly_bal_list[i][study_loan] * (1 + (foreclosure_charge_percent[study_loan] / 100)), 'closed loan of amount': principle_amount[study_loan], 'after #months': i - max_months_paid})
            deducted_saving = savings_list[i] - monthly_bal_list[i][study_loan] * (1 + (foreclosure_charge_percent[study_loan] / 100))
        
            while(len(study_loans) > 0):
                study_loan = my_arg(monthly_interest[i], 'max', set(range(no_of_loans)) - set(study_loans))
                settle_amount = monthly_bal_list[i][study_loan] * (1 + (foreclosure_charge_percent[study_loan] / 100))
                if deducted_saving > settle_amount:
                    study_loans.remove(study_loan)
                    marked_loans.append(study_loan)
                    results.append({'comments': 'preclosure', '#loan': study_loan, 'cumulative saving': deducted_saving, 'payment amount': monthly_bal_list[i][study_loan] * (1 + (foreclosure_charge_percent[study_loan] / 100)), 'closed loan of amount': principle_amount[study_loan], 'after #months': i - max_months_paid})
                    deducted_saving = deducted_saving - settle_amount
                else:
                    break
            
            for x in range(i+1, len(savings_list)):
                savings_list[x] = savings_list[x] - (savings_list[i] - deducted_saving)
                for loan in marked_loans:
                    monthly_interest[x][loan] = 0
                    monthly_bal_list[x][loan] = 0
            closed_loans.extend(marked_loans)
            
        if len(closed_loans) == no_of_loans:
            break
    return monthly_bal_list, monthly_interest, savings_list, results

In [18]:
def predict_foreclosure(principle_amount=[25000],
                        interest_rate=[36],
                        months_paid=[1],
                        tenure=[12],
                        fixed_monthly_saving=2000,
                        future_months_saving_plan={},
                        total_saved_yet=1000,
                        foreclosure_charge_percent=[5],
                        criterion='lowest_amount_first'):
    
    emi, loan_variables, no_of_loans = fixed_params(principle_amount, interest_rate, tenure)
    print('emi: ', emi)
#     print('loan_variables: ', loan_variables)

    monthly_bal_list, monthly_interest, savings_list, total_no_of_payment_months, max_months_paid = monthly_data(loan_variables, no_of_loans, months_paid, tenure, total_saved_yet, future_months_saving_plan, fixed_monthly_saving)
    
#     print('\n')
#     print('savings_list:\n', savings_list)
#     print('\n')
#     print('monthly_bal_list:\n', monthly_bal_list)
#     print('\n')
#     print('monthly_interest:\n', monthly_interest) 
    
    updated_monthly_bal_list, updated_monthly_interest, updated_savings_list, results = analysis(max_months_paid, monthly_bal_list, monthly_interest, savings_list, total_no_of_payment_months, no_of_loans, months_paid, criterion, foreclosure_charge_percent, principle_amount)
    
#     print('\n')
#     print('updated_savings_list:\n', updated_savings_list)
#     print('\n')
#     print('updated_monthly_bal_list:\n', updated_monthly_bal_list)
#     print(tabulate(updated_monthly_bal_list))
#     month = 0
#     for x in updated_monthly_bal_list:
#         month += 1
#         print(month, x)
        
#     print('\n')
#     print('updated_monthly_interest:\n', updated_monthly_interest)
    print('\n')
    for x in results:
        pprint(x)
        
#     paid_total = 0
#     for loan in no_of_loans:
#         for result in results:
#             emi_total+=emi[result['#loan']]*(result['after #months']-1)+result['payment amount'] 

#     paid_total = 
#     foreclosure_totals =

In [19]:
from tabulate import tabulate
predict_foreclosure(principle_amount=[91000, 100000, 90000, 52020],
                    interest_rate=[10, 30, 12, 13.5],
                    months_paid=[2, 4, 5, 10],
                    tenure=[48, 18, 36, 12],
                    fixed_monthly_saving=2000,
                    foreclosure_charge_percent=[5, 5, 5, 5],
                    total_saved_yet=0,
#                     criterion='highest_interest_first',
                    future_months_saving_plan={}
                    )

emi:  [2307.995092562001, 6967.008054917951, 2989.287883156605, 4658.496466898311]


{'#loan': 3,
 'after #months': 2,
 'closed loan of amount': 52020,
 'comments': 'auto closure done in this month',
 'cumulative saving': 4000,
 'payment amount': 0}
{'#loan': 1,
 'after #months': 11,
 'closed loan of amount': 100000,
 'comments': 'preclosure',
 'cumulative saving': 22000,
 'payment amount': 20892.83612841548}
{'#loan': 2,
 'after #months': 23,
 'closed loan of amount': 90000,
 'comments': 'preclosure',
 'cumulative saving': 25107.16387158452,
 'payment amount': 24016.720968640515}
{'#loan': 0,
 'after #months': 36,
 'closed loan of amount': 91000,
 'comments': 'preclosure',
 'cumulative saving': 27090.442902944003,
 'payment amount': 23159.270104681786}
